In [1]:
from utils import get_grouped_df

df = get_grouped_df()
df.head()

/home/apprenant/Desktop/Predictif_des_Entrees_de_Films/notebooks/utils.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,title,entries,date,realisateur,budget,titre_non_modifie,film_id,nom_acteur,nom_compagnie,nom_genre
0,#chef,13798,2014-10-29,Jon Favreau,11000000,#Chef,1888,"John Leguizamo, Jon Favreau, Sofía Vergara, Em...","Altus Media, Fairview Entertainment, Prescience",Comédie
1,#piresoiree,66666,2017-08-02,Lucia Aniello,20000000,# Pire Soirée,2260,"Scarlett Johansson, Jillian Bell, Ilana Glazer...","Paulilu Productions, Sony Pictures, Columbia P...","Drame, Comédie"
2,(500)joursensemble,82432,2009-09-30,Marc Webb,7500000,(500) jours ensemble,844,"Zooey Deschanel, Joseph Gordon-Levitt, Chloë G...","Fox Searchlight Pictures, Watermark Pictures, ...","Comédie, Drame, Romance"
3,100%cachemire,197125,2013-12-11,Valérie Lemercier,1400000,100% cachemire,1680,"Nanou Garcia, Gilles Lellouche, Valérie Lemerc...","SCOPE Pictures, Rectangle Productions",Comédie
4,10000,474853,2008-03-12,Roland Emmerich,105000000,10 000,606,"Cliff Curtis, Steven Strait, Nathanael Baring,...","Centropolis Entertainment, Legendary Pictures,...","Aventure, Action, Drame, Fantastique"
